In [1]:
from PIL import Image
import sys
import numpy as np
from src.helper import readImage, RGBtoGRAY, makeShape8Divisible, changeRange, removePadding, sharpenImage,saveImage
import matplotlib.pyplot as plt
from scipy.fftpack import dct, dst
from src.seam import removeSeam, insertSeam
from src.transforms import DCT, DST 
import os
from pathlib import Path
import matplotlib.pyplot as plt


In [2]:
def seamCarving(scaledImage, ImportanceMap, alpha=2, number_w=150, number_h=150):
    h,w=ImportanceMap.shape
    desired_w = w-number_w
    desired_h = h-number_h
    current_h, current_w = ImportanceMap.shape
    # saveProgress=True
    # first we will remove seams as it will help in complexity later if seams are to be inserted in other axis
    #row / column removal
    while(desired_w<current_w):
        scaledImage, ImportanceMap = removeSeam(ImportanceMap, scaledImage, removeCount=number_w, alpha=alpha, axis=0, save=False)
        current_h, current_w=ImportanceMap.shape

    while(desired_h<current_h):
        scaledImage, ImportanceMap = removeSeam(ImportanceMap, scaledImage, removeCount=number_h, alpha=alpha, axis=1,save=False)
        current_h, current_w=ImportanceMap.shape
#     return scaledImage,ImportanceMap
    return scaledImage,ImportanceMap

In [6]:
from tqdm import tqdm
import multiprocessing
import os
import numpy as np
from PIL import Image

def parallelSeamCarving(name):
    # print(f'{name}---------------')
    img_RGB = np.array(Image.open(os.path.join(Path, name)))

    cosineTransform = DCT()
    sineTransform = DST()

    # Convert RGB image to grayscale
    img_gray = RGBtoGRAY(img_RGB)

    # Perform DCT
    DCT_transformed_img = cosineTransform.transform(cosineTransform.transform(img_gray).T).T

    partialX = sineTransform.transform(cosineTransform.transform(DCT_transformed_img))
    partialY = sineTransform.transform(cosineTransform.transform(DCT_transformed_img.T)).T

    # Calculate Importance map based on partial derivatives
    ImportanceMap = np.abs(partialX) + np.abs(partialY)
    ImportanceMap = 255 * (ImportanceMap - ImportanceMap.min()) / (ImportanceMap.max() - ImportanceMap.min())
    
    # Convert to uint8
    ImportanceMap = ImportanceMap.astype(np.uint8)
    
    # Save the Importance Map
    tempIM = Image.fromarray(ImportanceMap)
    tempIM.save(os.path.join(importanceMapPath, name))
    
    scaledImage = img_RGB
    
    # Perform the seam carving
    totalSeamRemoved = 0
    for j in removePixels:
        scaledImage, ImportanceMap = seamCarving(scaledImage, ImportanceMap, alpha=1, number_w=j, number_h=j)
        totalSeamRemoved += j
        tempRGB = Image.fromarray(scaledImage.astype(np.uint8))
        tempRGB.save(os.path.join(TargetPath, name))

    return name  # Return the name for progress tracking


Path = "/home/scai/msr/aiy227514/scratch/Assignment783/Part2/Dataset/Inputs"
TargetPath = '/home/scai/msr/aiy227514/scratch/Assignment783/Part2/Dataset/NewGroundTruth'
importanceMapPath = '/home/scai/msr/aiy227514/scratch/Assignment783/Part2/Dataset/ImportanceMaps'
names = os.listdir(Path)

sigma = 25
weight = 0.75
removePixels = [128]  # Number of seams removed one after another

with multiprocessing.Pool(processes=8) as executor:
    # Wrap executor.map with tqdm to show a progress bar
    for _ in tqdm(executor.imap_unordered(parallelSeamCarving, names), total=len(names)):
        pass  # tqdm automatically updates with each result
